In [ ]:
# =======================================================
# Open Food Facts — Data Cleaning & Transformation
# =======================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---------- Style ----------
sns.set_style("whitegrid")
COLOR_MAIN = "#1b5e20"   # dark green
COLOR_ALT  = "#66bb6a"   # medium green
COLOR_ACC  = "#2e7d32"   # accent green
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "axes.titlesize": 14,
    "axes.titleweight": "bold",
    "axes.labelsize": 12,
    "xtick.labelsize": 10.5,
    "ytick.labelsize": 10.5,
})

OUT_DIR = "figures_clean_demo"
os.makedirs(OUT_DIR, exist_ok=True)

def save_with_caption(fig, filename, caption, bottom_pad=0.06, top_pad=0.94):
    """Embed a centered caption below the chart and save."""
    fig.text(0.5, 0.02, caption, ha="center", fontsize=10.5, color="#333333", wrap=True)
    plt.tight_layout(rect=[0.02, bottom_pad, 0.98, top_pad])
    fig.savefig(os.path.join(OUT_DIR, filename), dpi=300, bbox_inches="tight")
    plt.close(fig)

# ---------- 1) Load a 5,000-row sample ----------
URL = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"
df = pd.read_csv(URL, sep="\t", compression="gzip", low_memory=False, nrows=5000)

# ---------- 2) Select relevant columns ----------
cols = [
    "product_name", "brands", "categories", "labels",
    "energy-kcal_100g", "sugars_100g", "fat_100g",
    "proteins_100g", "salt_100g"
]
df = df[[c for c in cols if c in df.columns]].copy()

# ---------- 3) Measure missingness BEFORE ----------
missing_before = df.isna().mean() * 100
overall_before = missing_before.mean()

# ---------- 4) Cleaning: text normalization + numeric coercion/imputation ----------
# Text normalization (lowercase, trim). Preserve true NaN (avoid literal "nan" as text).
for c in ["product_name", "brands", "categories", "labels"]:
    if c in df.columns:
        s = df[c].astype("string")
        s = s.str.strip().str.lower()
        s = s.where(~s.eq("nan"), np.nan)  # turn literal "nan" into real NaN
        df[c] = s

# Numeric coercion + median imputation
num_cols = [c for c in ["energy-kcal_100g", "sugars_100g", "fat_100g", "proteins_100g", "salt_100g"] if c in df.columns]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
    df[c] = df[c].fillna(df[c].median(skipna=True))

# ---------- 5) Feature engineering ----------
df["is_vegan"]   = df["labels"].str.contains("vegan", case=False, na=False) if "labels" in df.columns else False
df["is_organic"] = df["labels"].str.contains("organic|bio", case=False, na=False) if "labels" in df.columns else False

# Macro-category (coarse, readable groups)
def macro_category(cat):
    if pd.isna(cat): return np.nan
    c = str(cat).lower()
    if any(k in c for k in ["drink","juice","coffee","tea","water"]): return "Beverages"
    if any(k in c for k in ["snack","chocolate","biscuit","cookie","dessert"]): return "Snacks & Sweets"
    if any(k in c for k in ["fruit","vegetable","bean","seed","nut","cereal","grain"]): return "Plant-based & Produce"
    if any(k in c for k in ["meat","fish","poultry","pork","beef","egg","ham"]): return "Animal-based & Meat"
    if any(k in c for k in ["milk","cheese","yogurt","butter","dairy"]): return "Dairy & Alternatives"
    if any(k in c for k in ["bread","pasta","flour","cake","bakery","rice"]): return "Bakery & Grains"
    if any(k in c for k in ["sauce","oil","spice","seasoning","condiment","ketchup","mayo"]): return "Condiments & Oils"
    if any(k in c for k in ["frozen","ready","prepared","instant","canned"]): return "Prepared & Convenience"
    if any(k in c for k in ["industrial","additive","powder","mix","ultra-processed"]): return "Processed & Industrial Goods"
    return "Other / Unclassified"

df["macro_category"] = df["categories"].apply(macro_category) if "categories" in df.columns else np.nan

# ---------- 6) Measure missingness AFTER ----------
missing_after = df.isna().mean() * 100
overall_after = missing_after.mean()

# ---------- 7) FIGURE — Overall missingness (Before vs After) ----------
fig, ax = plt.subplots(figsize=(5.6, 4.6))
vals = [overall_before, overall_after]
ax.bar(["Before", "After"], vals, color=[COLOR_ALT, COLOR_MAIN], edgecolor="black")
ax.set_title("Overall Missingness — Before vs After", pad=10)
ax.set_ylabel("Average Missing (%)")
for i, h in enumerate(vals):
    ax.text(i, h + max(0.6, h*0.03), f"{h:.1f}%", ha="center", va="bottom", fontweight="bold")
ax.grid(True, axis="y", alpha=0.35)
caption = ("Figure 1. Average missingness decreased after text normalization and median imputation, "
           "improving data completeness across the selected variables.")
save_with_caption(fig, "missingness_overall.png", caption)

# ---------- 8) FIGURE — Column-level reduction (Top 8) ----------
delta = (missing_before - missing_after).sort_values(ascending=False).head(8)
fig, ax = plt.subplots(figsize=(7.2, 4.8))
ax.barh(delta.index, delta.values, color=COLOR_MAIN, alpha=0.95)
ax.set_title("Reduction in Missingness per Column", pad=10)
ax.set_xlabel("Reduction (percentage points)")
for i, v in enumerate(delta.values):
    ax.text(v + 0.5, i, f"-{v:.1f}pp", va="center", fontsize=9)
plt.gca().invert_yaxis()  # largest on top
ax.grid(True, axis="x", alpha=0.3)
caption = ("Figure 2. Variables with the greatest reduction in missingness after preprocessing. "
           "Text fields benefited from normalization; numeric fields from median imputation.")
save_with_caption(fig, "missingness_gain.png", caption)

# ---------- 9) FIGURE — Macro-categories (excl. 'Other') ----------
if "macro_category" in df.columns:
    counts = df["macro_category"].value_counts()
    counts_wo = counts.drop(labels=["Other / Unclassified"], errors="ignore")
    counts_wo = counts_wo.head(10)
    if len(counts_wo) > 0:
        fig, ax = plt.subplots(figsize=(8.8, 5.2))
        sns.barplot(x=counts_wo.values, y=counts_wo.index, color=COLOR_ACC, edgecolor="black", ax=ax)
        ax.set_title("Top Macro-Categories (excluding 'Other / Unclassified')", pad=10)
        ax.set_xlabel("Number of Products"); ax.set_ylabel("Macro Category")
        for i, v in enumerate(counts_wo.values):
            ax.text(v + 5, i, f"{int(v)}", va="center", fontsize=9.5)
        ax.grid(True, axis="x", alpha=0.3)
        caption = ("Figure 3. Category engineering reduces raw label noise into interpretable macro-groups, "
                   "supporting clearer descriptive analysis and segmentation.")
        save_with_caption(fig, "macro_categories_excl_other.png", caption)

# ---------- 10) FIGURE — Feature results (Vegan vs Organic) ----------
vegan_pct   = df["is_vegan"].mean() * 100 if "is_vegan" in df.columns else 0.0
organic_pct = df["is_organic"].mean() * 100 if "is_organic" in df.columns else 0.0

fig, ax = plt.subplots(figsize=(5.0, 4.6))
bars = ax.bar(["Vegan", "Organic"], [vegan_pct, organic_pct],
              color=[COLOR_ALT, COLOR_MAIN], edgecolor="black")
for i, b in enumerate(bars):
    h = b.get_height()
    ax.text(i, h + 0.4, f"{h:.1f}%", ha="center", fontweight="bold")
ax.set_title("Share of Vegan and Organic Products", pad=10)
ax.set_ylabel("Percentage of Products")
ax.grid(True, axis="y", alpha=0.35)
caption = ("Figure 4. Derived binary features from label text enable simple product segmentation "
           "by sustainability-related signals (vegan and organic).")
save_with_caption(fig, "vegan_organic_share.png", caption)

# ---------- Done ----------
print("Figures saved to:", os.path.abspath(OUT_DIR))
for f in ["missingness_overall.png", "missingness_gain.png",
          "macro_categories_excl_other.png", "vegan_organic_share.png"]:
    p = os.path.join(OUT_DIR, f)
    if os.path.exists(p): print(" -", p)

Figures saved to: c:\Users\Laiss\OneDrive\Área de Trabalho\Portfolio\food_cleaning\figures_clean_demo
 - figures_clean_demo\missingness_overall.png
 - figures_clean_demo\missingness_gain.png
 - figures_clean_demo\macro_categories_excl_other.png
 - figures_clean_demo\vegan_organic_share.png
